# Retrieval Augmented Generation

Table of Contents
1. Idea
2. Naive Implementation
3. Graph Implementation

In [31]:
from language_models.proxy_client import BTPProxyClient
from language_models.agents.react import ReActAgent
from language_models.agents.chain import AgentChain
from language_models.models.llm import OpenAILanguageModel
from language_models.models.embedding import SentenceTransformerEmbeddingModel
from language_models.retrievers import BasicRetriever, CoALARetriever, ContextualCompressionRetriever
from language_models.retrievers.utils import split_documents
from language_models.vector_stores import FAISSVectorStore
from language_models.settings import settings
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from utils import load_docs_from_json, save_docs_to_json
from pydantic import BaseModel, Field
from numpy import dot
from numpy.linalg import norm
from bs4 import BeautifulSoup as Soup

In [15]:
proxy_client = BTPProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

In [16]:
try:
    documents = load_docs_from_json('data.json')
except:
    documents = []
    urls = [
        'https://www.apple.com/iphone/',
        'https://www.apple.com/ipad/',
        'https://www.apple.com/watch/',
        'https://www.apple.com/airpods/',
        'https://www.apple.com/mac/',
        'https://store.google.com/us/category/phones?hl=en-US',
        'https://store.google.com/us/category/watches_trackers?hl=en-US',
        'https://store.google.com/us/product/pixel_tablet?hl=en-US',
        'https://store.google.com/us/category/earbuds?hl=en-US',
        'https://www.samsung.com/us/smartphones/',
        'https://www.samsung.com/us/watches/',
        'https://www.samsung.com/us/tablets/',
        'https://www.samsung.com/us/mobile-audio/',
        'https://www.samsung.com/us/galaxybooks/',
    ]
    for url in urls:
        loader = RecursiveUrlLoader(url=url, max_depth=5, extractor=lambda x: Soup(x, "html.parser").text)
        docs = loader.load()
        documents += docs
    save_docs_to_json(documents, 'data.json')

## Idea

In [17]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-4",
    max_tokens=256,
    temperature=0.0,
)

In [18]:
class Output(BaseModel):
    content: str = Field(description="The final answer.")

agent = ReActAgent.create(
    llm=llm,
    system_prompt="",
    task_prompt="{question}",
    task_prompt_variables=["question"],
    output_format=Output,
    iterations=3,
)

In [19]:
response = agent.invoke({"question": "Tell me about the iPhone 15 pro and pro max."})

13/05/24 09:52:40 INFO Prompt:
Tell me about the iPhone 15 pro and pro max.
13/05/24 09:52:49 INFO Raw response:
{
  "thought": "The user is asking about the iPhone 15 Pro and Pro Max, but as of now, these models do not exist. The latest models from Apple are the iPhone 13 Pro and Pro Max. I should inform the user about this.",
  "tool": "Final Answer",
  "tool_input": {"content": "As of now, the iPhone 15 Pro and Pro Max do not exist. The latest models from Apple are the iPhone 13 Pro and Pro Max. Please check the official Apple website or authorized retailers for the most recent information."}
}
13/05/24 09:52:49 INFO Thought:
The user is asking about the iPhone 15 Pro and Pro Max, but as of now, these models do not exist. The latest models from Apple are the iPhone 13 Pro and Pro Max. I should inform the user about this.
13/05/24 09:52:49 INFO Final answer:
{'content': 'As of now, the iPhone 15 Pro and Pro Max do not exist. The latest models from Apple are the iPhone 13 Pro and Pro 

In [20]:
print(response.final_answer["content"])

As of now, the iPhone 15 Pro and Pro Max do not exist. The latest models from Apple are the iPhone 13 Pro and Pro Max. Please check the official Apple website or authorized retailers for the most recent information.


In [21]:
question = """Tell me about the iPhone 15 pro and pro max.

Use this context to answer the question:
- iPhone 15 Pro and Pro Max are the latest models.
- Available in Natural, Blue, White, and Black titanium.
- A17 Pro chip with 6-core GPU for fast performance.
- Pro camera: 48MP Main, 3x or 5x Telephoto, Ultra Wide.
- Up to 29 hours of video playback.
- ProMotion and Always-On display.
- Titanium back with textured matte glass.
- USB-C for 20x faster transfers.
- Emergency SOS, Crash Detection, Roadside Assistance via satellite."""

response = agent.invoke({"question": question})

13/05/24 09:52:49 INFO Prompt:
Tell me about the iPhone 15 pro and pro max.

Use this context to answer the question:
- iPhone 15 Pro and Pro Max are the latest models.
- Available in Natural, Blue, White, and Black titanium.
- A17 Pro chip with 6-core GPU for fast performance.
- Pro camera: 48MP Main, 3x or 5x Telephoto, Ultra Wide.
- Up to 29 hours of video playback.
- ProMotion and Always-On display.
- Titanium back with textured matte glass.
- USB-C for 20x faster transfers.
- Emergency SOS, Crash Detection, Roadside Assistance via satellite.
13/05/24 09:52:58 INFO Raw response:
{
  "thought": "I now know what to respond",
  "tool": "Final Answer",
  "tool_input": {
    "content": "The iPhone 15 Pro and Pro Max are the latest models from Apple. They are available in Natural, Blue, White, and Black titanium. They are powered by the A17 Pro chip with a 6-core GPU for fast performance. The Pro camera system includes a 48MP Main camera, a 3x or 5x Telephoto camera, and an Ultra Wide ca

In [22]:
print(response.final_answer["content"])

The iPhone 15 Pro and Pro Max are the latest models from Apple. They are available in Natural, Blue, White, and Black titanium. They are powered by the A17 Pro chip with a 6-core GPU for fast performance. The Pro camera system includes a 48MP Main camera, a 3x or 5x Telephoto camera, and an Ultra Wide camera. They offer up to 29 hours of video playback. They also feature ProMotion and an Always-On display. The back is made of titanium with a textured matte glass finish. They use USB-C for 20x faster data transfers. They also have safety features like Emergency SOS, Crash Detection, and Roadside Assistance via satellite.


In [23]:
embedding_model = SentenceTransformerEmbeddingModel(model="all-MiniLM-L6-v2")

In [24]:
query1 = "Tell me about the iPhone 15 pro and pro max."
embedding1 = embedding_model.embed_query(query1)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
print(embedding1)

[-0.04976186528801918, 0.024698281660676003, 0.024233346804976463, -0.08842168003320694, 0.0374833419919014, -0.01828034222126007, 0.004888656549155712, 0.11331886053085327, 0.016784582287073135, 0.08297941088676453, 0.10016509890556335, 0.05813554301857948, 0.0017647211207076907, 0.04127701371908188, 0.010860818438231945, 0.03729262202978134, 0.1335870325565338, -0.08417602628469467, -0.05983923748135567, -0.10192283987998962, -0.01667306385934353, -0.07314975559711456, 0.06727840006351471, 0.049850501120090485, 0.04839256778359413, 0.05804368853569031, -0.06915360689163208, -0.009908149018883705, -0.03353537619113922, -0.0551525242626667, 0.02574399672448635, -0.02915993705391884, 0.07582292705774307, -0.013736454769968987, -0.013520478270947933, -0.08669689297676086, 0.0033832842018455267, -0.023326121270656586, 0.008765621110796928, -0.06883351504802704, -0.03609778359532356, -0.0032673256937414408, -0.056928034871816635, 0.052639756351709366, 0.05331815406680107, -0.01249685883522

In [26]:
query2 = """- iPhone 15 Pro and Pro Max are the latest models.
- Available in Natural, Blue, White, and Black titanium.
- A17 Pro chip with 6-core GPU for fast performance.
- Pro camera: 48MP Main, 3x or 5x Telephoto, Ultra Wide.
- Up to 29 hours of video playback.
- ProMotion and Always-On display.
- Titanium back with textured matte glass.
- USB-C for 20x faster transfers.
- Emergency SOS, Crash Detection, Roadside Assistance via satellite."""
embedding2 = embedding_model.embed_query(query2)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
cosine_similarity = dot(embedding1, embedding2) / (norm(embedding1) * norm(embedding2))
print(f"Cosine similarity: {cosine_similarity}")

Cosine similarity: 0.738208049156622


## Naive Implementation

## Graph Implementation